In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d pdavpoojan/the-rvlcdip-dataset-test

Dataset URL: https://www.kaggle.com/datasets/pdavpoojan/the-rvlcdip-dataset-test
License(s): unknown
100% 3.62G/3.62G [00:46<00:00, 145MB/s]
100% 3.62G/3.62G [00:46<00:00, 83.9MB/s]


In [ ]:
!unzip '/content/the-rvlcdip-dataset-test.zip'

Streaming output truncated to the last 5000 lines.
  inflating: test/scientific_publication/PUBLICATIONS040137-0.tif  
  inflating: test/scientific_publication/PUBLICATIONS041468-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS041570-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS041737-1.tif  
  inflating: test/scientific_publication/PUBLICATIONS042046-2.tif  
  inflating: test/scientific_publication/PUBLICATIONS042251A-.tif  
  inflating: test/scientific_publication/PUBLICATIONS042455-2.tif  
  inflating: test/scientific_publication/PUBLICATIONS044016-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044594-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044641-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044730-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS044895-4.tif  
  inflating: test/scientific_publication/PUBLICATIONS045235-5.tif  
  inflating: test/scientific_publication/PUBLICATIONS045310-5.tif

In [ ]:
!pip install torch
!pip install torchvision
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,646 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import os
import pytesseract
from PIL import Image
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

In [ ]:
def extract_text_from_image(args):
    """
    Simple text extraction using Tesseract directly on original image
    """
    image_path, output_dir = args
    try:
        # Open image directly with PIL
        image = Image.open(image_path)

        # Extract text
        extracted_text = pytesseract.image_to_string(image)

        return {
            'filename': os.path.basename(image_path),
            'extracted_text': extracted_text.strip()
        }

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return {
            'filename': os.path.basename(image_path),
            'extracted_text': "",
            'error': str(e)
        }

In [ ]:
def process_directory_parallel(input_dir, output_dir, max_workers=None):
    """
    Process all images in parallel
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get list of all images
    image_files = [
        f for f in os.listdir(input_dir)
        if f.lower().endswith(('.tif'))
    ]

    # Convert to full paths
    image_files = [os.path.join(input_dir, f) for f in image_files]

    # If max_workers not specified, use number of CPU cores
    if max_workers is None:
        max_workers = multiprocessing.cpu_count()

    # Prepare arguments for parallel processing
    process_args = [(f, output_dir) for f in image_files]

    # Process images in parallel
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(extract_text_from_image, process_args))

    # Create DataFrame and save results
    df = pd.DataFrame(results)
    results_path = os.path.join(output_dir, "extraction_results.csv")
    df.to_csv(results_path, index=False)

    return df

In [ ]:
if __name__ == "__main__":
    input_directory = "/content/test/invoice"  # Update with your input directory
    output_directory = "/content/processed_invoices"  # Update with your output directory

    # Get number of CPU cores for parallel processing
    num_cores = multiprocessing.cpu_count()
    print(f"Processing using {num_cores} CPU cores")

    # Process all images in parallel
    results_df = process_directory_parallel(
        input_directory,
        output_directory,
        max_workers=num_cores
    )

    print("Processing complete. Results saved to extraction_results.csv")

Processing using 2 CPU cores
Processing complete. Results saved to extraction_results.csv
